In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data-input.csv')
print(df)

In [ ]:
df.to_csv('data.csv')

In [ ]:
df.head()

Found how to do most of this from "https://realpython.com/python-data-cleaning-numpy-pandas/"

Just removing some of the columns that we dont really care about or have overlap with other columns.

In [ ]:
to_drop = ['keywords',
            'homepage',
            'production_countries',
            'spoken_languages',
            'tagline',
            'status',
            'original_title']

new_names =  {'original_language': 'language'}
df.drop(to_drop, inplace=True, axis=1)
df.rename(columns=new_names, inplace=True)

# There was a bunch of rows that had 0 runtime/revenue that were skewing all my data. hopefully this helps?
df = df[df.revenue != 0]
df = df[df.runtime != 0]
df = df[df.vote_count != 0]

df.describe()

In [ ]:
df.head()

In [ ]:
# took from here https://www.interviewqs.com/ddi-code-snippets/extract-month-year-pandas
# getting year month day for future calculations potentially
import datetime

df['year'] = pd.DatetimeIndex(df['release_date']).year
df['month'] = pd.DatetimeIndex(df['release_date']).month
df['day'] = pd.DatetimeIndex(df['release_date']).day

df.head()

In [ ]:
# I had a value in my csv in 1927 with a 93 million dollar budget. I looked it up and this was incorrect
# So I needed to clean this datapoint and remove it from the csv
df = df[df.year != 1927]

In [ ]:
df.to_csv('data-cleaned.csv')

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

data = pd.read_csv(r'data-cleaned.csv')

In [ ]:
plot1 = sns.scatterplot(x="budget", y="revenue", data=data)

plt.show()

# wouldnt save the jpg when I did it same as below for some reason.
fig = plot1.get_figure()
fig.savefig("plot1.jpg")

In [ ]:
plot2 = sns.lineplot(x="year", y="budget", data=data)

plt.show

plt.savefig("plot2.jpg")

In [ ]:
plot3 = sns.displot(data['month'])

plt.show

plt.savefig("plot3.jpg")

In [ ]:
data.head()

In [ ]:
import tensorflow as tf

tf.random.set_seed(1234)

# I dont think this is right but im dropping the columns I dont need now just so I can split the input/output values

to_drop = ['Unnamed: 0',
            'genres',
            'id',
            'language',
            'overview',
            'production_companies',
            'release_date',
            'title']

data.drop(to_drop, inplace=True, axis=1)
data.head()


In [ ]:
# Taken from https://stackoverflow.com/questions/43697240/how-can-i-split-a-dataset-from-a-csv-file-for-training-and-testing
train = data.sample(frac=0.8, random_state = np.random.RandomState())
test = data.loc[~data.index.isin(train.index)]

train.to_csv('main_train.csv', index=False)
test.to_csv('main_test.csv', index=False)

print("--Get data--")
y_test = test.month
x_test = test.drop('month', axis=1)

y_train = train.month
x_train = train.drop('month', axis=1)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

--Get data--
(2698, 8) (674, 8) (2698,) (674,)


In [ ]:
test.describe()

In [ ]:
print("--Make model--")
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, input_shape=(8,), activation="relu"),
  tf.keras.layers.Dense(32, activation="relu"),
  tf.keras.layers.Dense(13, activation = "softmax")
])

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
  initial_learning_rate, decay_steps=100000, decay_rate=0.9, staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("--Fit model--")
model.fit(x_train, y_train, epochs=50, verbose=2, batch_size = 20)

print("--Evaluate model--")
model_loss1, model_acc1 = model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = model.evaluate(x_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

--Make model--
--Fit model--
Epoch 1/50
135/135 - 1s - loss: 3788165.7500 - accuracy: 0.0875 - 1s/epoch - 9ms/step
Epoch 2/50
135/135 - 0s - loss: 741695.1875 - accuracy: 0.0856 - 285ms/epoch - 2ms/step
Epoch 3/50
135/135 - 0s - loss: 571967.3750 - accuracy: 0.0886 - 276ms/epoch - 2ms/step
Epoch 4/50
135/135 - 0s - loss: 547517.5000 - accuracy: 0.0967 - 279ms/epoch - 2ms/step
Epoch 5/50
135/135 - 0s - loss: 601266.0625 - accuracy: 0.0823 - 280ms/epoch - 2ms/step
Epoch 6/50
135/135 - 0s - loss: 506958.8125 - accuracy: 0.0823 - 287ms/epoch - 2ms/step
Epoch 7/50
135/135 - 0s - loss: 597651.9375 - accuracy: 0.0908 - 292ms/epoch - 2ms/step
Epoch 8/50
135/135 - 0s - loss: 448102.8125 - accuracy: 0.0923 - 274ms/epoch - 2ms/step
Epoch 9/50
135/135 - 0s - loss: 534155.6875 - accuracy: 0.0878 - 279ms/epoch - 2ms/step
Epoch 10/50
135/135 - 0s - loss: 454312.8438 - accuracy: 0.0927 - 291ms/epoch - 2ms/step
Epoch 11/50
135/135 - 0s - loss: 469340.5938 - accuracy: 0.0860 - 264ms/epoch - 2ms/step
Epo